In [52]:
import dlc_practical_prologue as plg
from torch import nn
from torch.nn import functional as F

In [107]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        nb_hidden = 10
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 4 * 4, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(self.conv2(x))
        x = F.relu(self.fc1(x.view(-1, 32 * 4 * 4)))
        x = self.fc2(x)
        return x

In [26]:
#1x14x14
#32x6x6
#32x4x4

In [87]:
N = 1000
train_input, train_target, train_classes, test_input, test_target, test_classes = plg.generate_pair_sets(N)

In [95]:
train_input_left = train_input[:,0,:,:].view(N,1,14,14)
train_classes_left = plg.convert_to_one_hot_labels(train_input, train_classes[:,0]).type(torch.LongTensor)
test_input_left = test_input[:,0,:,:].view(N,1,14,14)
test_classes_left = plg.convert_to_one_hot_labels(test_input, test_classes[:,0]).type(torch.LongTensor)

In [100]:
def train_model(model, train_input, train_target, mini_batch_size):
    
    if (torch.cuda.is_available()):
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
        
    model.to(device)
    train_input = train_input.to(device)
    train_target = train_target.to(device)
    
    criterion = nn.CrossEntropyLoss().to(device)
    eta = 1e-1
    epochs = 250
    optimizer = torch.optim.SGD(model.parameters(), lr = eta)
    
    for e in range(0, epochs):
        for input_data, target_data in zip(train_input.split(mini_batch_size), train_target.split(mini_batch_size)):
            output = model(input_data)
            #print(output)
            loss = criterion(output, target_data)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [103]:
def compute_nb_errors(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.max(1)
        for k in range(mini_batch_size):
            if target[b + k] != predicted_classes[k]:
                nb_errors = nb_errors + 1

    return nb_errors

In [108]:
model = Net()
train_model(model, train_input_left, train_classes[:,0], 100)

In [109]:
compute_nb_errors(model, train_input_left, test_classes[:,0], 100)

880